In [1]:
import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras

import tensorflow as tf

/apps/dslab/anaconda/python3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SIZE = 9

In [3]:
npz = np.load('models/train_vectors_9_9_r1.npz')
train_vectors = npz['train_vectors']
train_p = npz['train_p']
train_q = npz['train_q']

In [4]:
inp = Input(shape=(SIZE, SIZE, 1))

bn1 = BatchNormalization()(inp)
conv_1 = Convolution2D(64, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_uniform')(bn1)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_uniform')(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='valid', activation='relu',
                       kernel_initializer='random_uniform')(bn3)
bn4 = BatchNormalization()(conv_3)

flat = Flatten()(bn4)
turn_input = Input(shape=(1,), name='turn')
full = concatenate([flat, turn_input])

hidden = Dense(15, activation='relu', kernel_initializer='random_uniform')(full)
bn4 = BatchNormalization()(hidden)

out = Dense(1)(bn4)

q_model = Model(inputs=[inp, turn_input], outputs=out)
q_model.compile(loss=losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])

In [5]:
boards = np.array([board for board in train_vectors[:, 0]]).reshape(train_vectors.shape[0], SIZE, SIZE, 1)

In [ ]:
with tf.device('/gpu:0'):
    q_model.fit([boards, 
               train_vectors[:, 1]], 
              train_q, 
              epochs=10,
             validation_split=0.1)

Train on 89924 samples, validate on 9992 samples
Epoch 1/10
89924/89924 [==============================] - 23s 255us/step - loss: 0.0847 - mean_squared_error: 0.0847 - val_loss: 0.0572 - val_mean_squared_error: 0.0572
Epoch 2/10
89924/89924 [==============================] - 21s 228us/step - loss: 0.0401 - mean_squared_error: 0.0401 - val_loss: 0.0355 - val_mean_squared_error: 0.0355
Epoch 3/10
89924/89924 [==============================] - 20s 227us/step - loss: 0.0320 - mean_squared_error: 0.0320 - val_loss: 0.0321 - val_mean_squared_error: 0.0321
Epoch 4/10
89924/89924 [==============================] - 21s 229us/step - loss: 0.0274 - mean_squared_error: 0.0274 - val_loss: 0.0255 - val_mean_squared_error: 0.0255
Epoch 5/10
89924/89924 [==============================] - 20s 227us/step - loss: 0.0250 - mean_squared_error: 0.0250 - val_loss: 0.0261 - val_mean_squared_error: 0.0261
Epoch 6/10
89924/89924 [==============================] - 20s 223us/step - loss: 0.0233 - mean_squared_err

In [12]:
q_model.save('models/13_13_q_v3.net')

In [23]:
inp = Input(shape=(SIZE, SIZE, 1))

bn1 = BatchNormalization()(inp)
conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn1)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn3)
bn4 = BatchNormalization()(conv_3)
#conv_4 = Convolution2D(64, (3, 3), padding='same', activation='relu',
#                       kernel_initializer='random_uniform')(bn4)
#bn5 = BatchNormalization()(conv_4)
#conv_5 = Convolution2D(64, (3, 3), padding='same', activation='relu',
#                       kernel_initializer='random_uniform')(bn5)
#bn6 = BatchNormalization()(conv_5)

flat = Flatten()(bn4)
turn_input = Input(shape=(1,), name='turn')
full = concatenate([flat, turn_input])

hidden = Dense(15, activation='relu', kernel_initializer='random_uniform')(full)
bn4 = BatchNormalization()(hidden)

out = Dense(SIZE ** 2, activation='softmax')(bn4)

p_model = Model(inputs=[inp, turn_input], outputs=out)
p_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
with tf.device('/gpu:0'):
    p_model.fit([boards, 
               train_vectors[:, 1]], 
              train_p, 
              epochs=3,
             validation_split=0.1)

Train on 58464 samples, validate on 6496 samples
Epoch 1/3
58464/58464 [==============================] - 15s 256us/step - loss: 4.5833 - acc: 0.1080 - val_loss: 4.3583 - val_acc: 0.1404
Epoch 2/3
58464/58464 [==============================] - 13s 219us/step - loss: 4.1298 - acc: 0.1903 - val_loss: 4.2087 - val_acc: 0.1809
Epoch 3/3
58464/58464 [==============================] - 12s 213us/step - loss: 3.9583 - acc: 0.2184 - val_loss: 4.1568 - val_acc: 0.1940


In [58]:
p_model.save('models/13_13_p_v2.net')